In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv
import datetime
import time
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


## Scraping site using Selenium

In [36]:
# # Setting up Web Driver
# options = Options()
# options.add_argument("--headless=new")
# driver = webdriver.Chrome(options=options)
# driver.get('https://guide.wisc.edu/courses/')

# # Initializing empty dict for class dict 
# major_class_dict = {}

# course_list = driver.find_element(By.ID, "atozindex")
# subjects = course_list.find_elements(By.TAG_NAME, "a")

# # Gather all the URLs first
# links = [link.get_attribute('href') for link in subjects]
# names = [subject.text for subject in subjects]

# num_subjects = len(subjects)

# # Now visit each URL directly
# for i in range(num_subjects):
#     class_list = []
#     name = names[i].split(' (')[0]
#     print(name)
#     driver.get(links[i])
#     classes = WebDriverWait(driver, 20, ignored_exceptions=(NoSuchElementException, StaleElementReferenceException)).until(EC.presence_of_element_located((By.CLASS_NAME,  "sc_sccoursedescs")))
#     titles = classes.find_elements(By.CLASS_NAME, "courseblockcode")
#     credits = classes.find_elements(By.CLASS_NAME, "courseblockcredits")
#     courses = classes.find_elements(By.CLASS_NAME,  "courseblock")
#     num_courses = len(courses)
#     for j in range(num_courses):
#         class_dict = {}
#         class_dict['class'] = titles[j].text
#         class_dict['credits'] = int(credits[j].text[:1])

#         details = courses[j].find_elements(By.CLASS_NAME, "courseblockextra")
#         num_details = len(details)
#         class_details = {}
#         for k in range(num_details):
#             inner_html = details[k].get_attribute('innerHTML')
#             soup = BeautifulSoup(inner_html, 'html.parser')
#             text = soup.get_text().split(': ')
#             label = text[0]
#             info = text[1]
#             if re.search('\u200b|\xa0', info):
#                 info = info.replace('\u200b', '').replace('\xa0', ' ')
#             class_details[label] = info
#         class_dict['class info'] = class_details
#         class_list.append(class_dict)
#     major_class_dict[name] = class_list

# driver.quit()

In [35]:
# data = major_class_dict
# # Open a new CSV file for writing
# with open('courses.csv', 'w', newline='') as file:
#     writer = csv.writer(file)

#     # Write the header
#     writer.writerow(['Department', 'Class', 'Credits', 'Requisites',  'Course Designation', 'Repeatable for Credit', 'Last Taught'])

#     # Iterate over the departments and their courses
#     for department, courses in data.items():
#         for course in courses:
#             class_info = course['class info']
#             # Extract data, providing default values if any key is missing
#             requisites = class_info.get('Requisites', '')
#             repeatable = class_info.get('Repeatable for Credit', '')
#             last_taught = class_info.get('Last Taught', '')
#             course_designation = class_info.get('Course Designation', '')

#             # Write the data to the CSV file
#             writer.writerow([department, course['class'], course['credits'], requisites, course_designation, repeatable, last_taught])


## Scraping site using Requests

In [7]:
# # Base URL
# base_url = 'https://guide.wisc.edu'
# start_url = 'https://guide.wisc.edu/courses/'

# # Fetch main page content
# response = requests.get(start_url)
# soup = BeautifulSoup(response.content, 'html.parser')

# # Initializing empty dict for class dict 
# major_class_dict = {}

# # Extract subjects
# subjects = soup.select('#atozindex a')

# # Loop through subjects
# for subject in subjects:
#     class_list = []
#     subject_url = base_url + subject['href']
#     subject_name = subject.text.split(' (')[0]
    
#     print(subject_name)
    
#     # Get subject page content
#     subject_response = requests.get(subject_url)
#     subject_soup = BeautifulSoup(subject_response.content, 'html.parser')
    
#     # Extract class details
#     courses = subject_soup.select('.courseblock')
    
#     for course in courses:
#         class_dict = {}
#         title = course.select_one('.courseblockcode').text
#         if re.search('\u200b|\xa0', title):
#             title = title.replace('\u200b', '').replace('\xa0', ' ')
#         credit = course.select_one('.courseblockcredits').text
        
#         class_dict['class'] = title
#         class_dict['credits'] = int(credit[:1])
        
#         details = course.select('.courseblockextra')
#         class_details = {}
        
#         for detail in details:
#             text = detail.text.split(': ')
#             label = text[0]
#             info = text[1]
            
#             if re.search('\u200b|\xa0', info):
#                 info = info.replace('\u200b', '').replace('\xa0', ' ')
#             class_details[label] = info
        
#         class_dict['class info'] = class_details
#         class_list.append(class_dict)
    
#     major_class_dict[subject_name] = class_list


## Writing dictionary to CSV

In [8]:
# data = major_class_dict
# # Open a new CSV file for writing
# with open('allcourses.csv', 'w', newline='') as file:
#     writer = csv.writer(file)

#     # Write the header
#     writer.writerow(['Department', 'Class', 'Credits', 'Requisites',  'Course Designation', 'Repeatable for Credit', 'Last Taught'])

#     # Iterate over the departments and their courses
#     for department, courses in data.items():
#         for course in courses:
#             class_info = course['class info']
#             # Extract data, providing default values if any key is missing
#             requisites = class_info.get('Requisites', '')
#             repeatable = class_info.get('Repeatable for Credit', '')
#             last_taught = class_info.get('Last Taught', '')
#             course_designation = class_info.get('Course Designation', '')

#             # Write the data to the CSV file
#             writer.writerow([department, course['class'], course['credits'], requisites, course_designation, repeatable, last_taught])


In [9]:
# full_df = pd.read_csv('allcourses.csv')
# requisites = full_df['Requisites']
# for requisite in requisites:
#     try:
#         print(requisite)
#         reqs = re.split(r'[,.;]', requisite)
#         reqs = [req.strip() for req in reqs]
#         print(reqs)
#     except TypeError as e:
#         continue

In [6]:
full_df = pd.read_csv('allcourses.csv')
df = full_df[full_df['Last Taught'].str.slice(-4) >= str(datetime.datetime.now().year-3)]
df = df.reset_index(drop=True)
df.to_csv('cleanedallcourses.csv')

In [37]:
start_url = 'https://guide.wisc.edu/explore-majors/'
base_url = 'https://guide.wisc.edu'

response = requests.get(start_url)

soup = BeautifulSoup(response.content, 'html.parser')
items = soup.select_one('#filter-items')
programs = items.select('li a')

computer = 'Computer Sciences, B.S.'
ie = 'Industrial Engineering, B.S.'
math = 'Mathematics, B.S.'

classes = [computer, ie, math]

for program in programs:
    title = program.select_one('.title').text
    if title not in classes:
        continue
    # testing with only BS degrees first
    # if not re.search(r'', title):
    # if not re.search(r'\s*B\.S\.\s*', title):
        # continue
    program_url = base_url + program['href']
    program_response = requests.get(program_url)
    extract_content = BeautifulSoup(program_response.content, 'html.parser')
    requirement_link = extract_content.select_one('#requirementstexttab a')['href']
    requirements_url = program_url + requirement_link
    requirement_response = requests.get(requirements_url)
    requirement_content = BeautifulSoup(requirement_response.content, 'html.parser')
    print(title)
    course_list = requirement_content.select('.sc_courselist')

    row_li = []
    for course in course_list:
        rows = course.select('tbody tr')
        print(course.text)
        for row in rows:
            credit = re.match(r'(.*)\((\d+(?:-\d+)?)\)', row.text)
            row_li.append(row.text)
    print(row_li)



Computer Sciences, B.S.
 Course List        Code Title Credits    COMP SCI/​MATH  240 Introduction to Discrete Mathematics3 COMP SCI/​E C E  252 Introduction to Computer Engineering3 COMP SCI 300   Programming II3 COMP SCI/​E C E  354   Machine Organization and Programming3 COMP SCI 400 Programming III3 Total Credits15 
 Course List        Code Title Credits    Complete one of these sequences:9-14 MATH 221& MATH 222   Calculus and Analytic Geometry 1and    Calculus and Analytic Geometry 2 MATH 171& MATH 217& MATH 222   Calculus with Algebra and Trigonometry Iand    Calculus with Algebra and Trigonometry IIand    Calculus and Analytic Geometry 2 MATH 275& MATH 276   Topics in Calculus Iand    Topics in Calculus II Total Credits9-14 
 Course List        Code Title Credits    Complete two courses for at least 6 credits:6-10 MATH 340 Elementary Matrix and Linear Algebra 1 or MATH 375  Topics in Multi-Variable Calculus and Linear Algebra STAT 324 Introductory Applied Statistics for Engineer

## Work to get schools what classes are in each school

In [18]:

# base_url = 'https://guide.wisc.edu'
# start_url = 'https://guide.wisc.edu/undergraduate'

# response = requests.get(start_url)

# soup = BeautifulSoup(response.content, 'html.parser')
# items = soup.select_one('#schoolsandcollegestextcontainer')
# schools = items.select('li a')

# schools_dict = {}
# for school in schools:
#     programs_list = []
#     name = school.text
#     school_url = base_url + school['href']
#     school_response = requests.get(school_url)
#     extract_content = BeautifulSoup(school_response.content, 'html.parser')
#     requirement_link = extract_content.select_one('#degreesmajorscertificatestexttab a')['href']
#     requirements_url = school_url + requirement_link
#     requirement_response = requests.get(requirements_url)
#     requirement_content = BeautifulSoup(requirement_response.content, 'html.parser')
#     school_links = requirement_content.select('.visual-sitemap a')
#     for link in school_links:
#         if not re.search(r'\s*B\.S\.\s*', link.text):
#             continue
#         programs_list.append((link.text).split(',')[0])
#         print((link.text).split(',')[0])
#     schools_dict[name] = programs_list
# schools_dict
    

In [15]:
# df2 = pd.DataFrame(list(schools_dict.items()), columns=['School', 'Program'])
# df2 = df2.explode('Program', ignore_index=True)
# df2

In [2]:
from graphviz import Graph